In [1]:
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time

In [2]:
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config["display.limit"] = 20
    
#schema = dj.schema('microns_ta3p100')
#ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema("microns_pinky")
pinky = dj.create_virtual_module("pinky","microns_pinky")


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class WholeAutoAnnotationsClusters3(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> pinky.PymeshfixDecimatedExcitatoryStitchedMesh
    ---
    vertices   : longblob     # label data for the vertices
    triangles  : longblob     # label data for the faces
    """
    
    def make(self, key):
        start_time = time.time()
        
        print("\n\n*****Starting Auto Labeling for " + str(key["segment_id"]) + "******")
        
        segment_id = key["segment_id"]

        #get the vertices and faces from datajoint
        # get the newly stitched mesh
        # get the original mesh

        
        verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & key).fetch1("vertices","triangles")


        #run the whole algorithm on the neuron to test
        verts_labels, faces_labels = extract_branches_whole_neuron(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                             triangles=faces,pymeshfix_Flag=False,
                             import_CGAL_Flag=False,
                             return_Only_Labels=True,
                             clusters=3,
                             smoothness=0.2)
        
        #insert the key
        self.insert1(dict(key,
                  vertices=verts_labels,
                  triangles=faces_labels),skip_duplicates=True)


        
        print(f"Finished Auto Labeling: {time.time() - start_time}")
        

In [4]:
start = time.time()
WholeAutoAnnotationsClusters3.populate(reserve_jobs=True)
print(time.time() - start)



*****Starting Auto Labeling for 648518346349496672******
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.10270094871520996
2) Staring: Generating CGAL segmentation for neuron
Done writing OFF file

Starting CGAL segmentation
Right before cgal segmentation, clusters = 3, smoothness = 0.2
1
Finished CGAL segmentation algorithm: 1107.563892841339
2) Finished: Generating CGAL segmentation for neuron: 1129.3178617954254
3) Staring: Generating Graph Structure and Identifying Soma
soma_index = 3
3) Finished: Generating Graph Structure and Identifying Soma: 0.2639329433441162
5) Staring: Finding Apical Index
Soma Index = 3
Soma Connections = [9, 16, 23, 15, 25, 30, 107, 27, 58, 67, 103, 117, 104]
soma_80_percent = 225709.9828125
Debugging the axon filter
[(9, 222808.53125), (16, 245732.21875), (23, 249933.0625), (15, 236272.015625), (25, 223846.359375), (30, 226765.234375), (107, 243847.09375), (27, 22416

In [5]:
#(schema.jobs & "table_name='__whole_auto_annotations_clusters3'").delete()

In [6]:
#schema.jobs